## Przygotowanie danych, foundational foods 
czyli te "defaultowe" jak np "jabłko", "wołowina" itp  
branded zajmę się później, bo to wielogigabajtowy chonker

In [2]:
import json
import pandas as pd

with open("raw_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Spłaszczenie listy FoundationFoods ( bo tam wszystko jest jakby zagnieżdżone w jednym "foundation foods")
df = pd.json_normalize(data['FoundationFoods'])
df.head()



,foodClass,description,foodNutrients,foodAttributes,nutrientConversionFactors,isHistoricalReference,ndbNumber,dataType,fdcId,foodPortions,publicationDate,inputFoods,foodCategory.description,scientificName
0,FinalFood,"Hummus, commercial","[{'type': 'FoodNutrient', 'id': 2219707, 'nutr...",[],"[{'type': '.CalorieConversionFactor', 'protein...",False,16158,Foundation,321358,"[{'id': 118804, 'value': 2.0, 'measureUnit': {...",4/1/2019,"[{'id': 10428, 'foodDescription': 'HUMMUS, SAB...",Legumes and Legume Products,NaN
1,FinalFood,"Tomatoes, grape, raw","[{'type': 'FoodNutrient', 'id': 2219983, 'nutr...",[],"[{'type': '.ProteinConversionFactor', 'value':...",False,100147,Foundation,321360,"[{'id': 118808, 'value': 5.0, 'measureUnit': {...",4/1/2019,"[{'id': 10480, 'foodDescription': 'TOMATOES, G...",Vegetables and Vegetable Products,Solanum lycopersicum
2,FinalFood,"Beans, snap, green, canned, regular pack, drai...","[{'type': 'FoodNutrient', 'id': 2220526, 'nutr...",[],"[{'type': '.CalorieConversionFactor', 'protein...",False,11056,Foundation,321611,"[{'id': 118859, 'value': 1.0, 'measureUnit': {...",4/1/2019,"[{'id': 10515, 'foodDescription': 'BEANS, SNAP...",Vegetables and Vegetable Products,NaN
3,FinalFood,"Frankfurter, beef, unheated","[{'type': 'FoodNutrient', 'id': 2227684, 'nutr...",[],"[{'type': '.CalorieConversionFactor', 'protein...",False,7022,Foundation,323121,"[{'id': 118987, 'value': 1.0, 'measureUnit': {...",4/1/2019,"[{'id': 10612, 'foodDescription': 'HOT DOGS, B...",Sausages and Luncheon Meats,NaN
4,FinalFood,"Nuts, almonds, dry roasted, with salt added","[{'type': 'FoodNutrient', 'id': 2228441, 'nutr...",[],"[{'type': '.ProteinConversionFactor', 'value':...",False,12563,Foundation,323294,"[{'id': 119012, 'value': 1.0, 'measureUnit': {...",4/1/2019,"[{'id': 10625, 'foodDescription': 'ALMONDS, DR...",Nut and Seed Products,NaN


### print etykietek 

In [4]:
from pprint import pprint

flat_keys = set()
nutrient_keys = set()
derivation_keys = set()
source_keys = set()

for food in data["FoundationFoods"]:
    for item in food.get("foodNutrients", []):
        flat_keys.update(item.keys())

        if isinstance(item.get("nutrient"), dict):
            nutrient_keys.update(item["nutrient"].keys())

        if isinstance(item.get("foodNutrientDerivation"), dict):
            derivation_keys.update(item["foodNutrientDerivation"].keys())
            source = item["foodNutrientDerivation"].get("foodNutrientSource")
            if isinstance(source, dict):
                source_keys.update(source.keys())

print("🔹 Główne pola w foodNutrients:")
pprint(sorted(flat_keys))

print("\n🔸 Pola w nutrient:")
pprint(sorted(nutrient_keys))

print("\n🔸 Pola w foodNutrientDerivation:")
pprint(sorted(derivation_keys))

print("\n🔸 Pola w foodNutrientDerivation.foodNutrientSource:")
pprint(sorted(source_keys))


🔹 Główne pola w foodNutrients:
['amount',
 'dataPoints',
 'foodNutrientDerivation',
 'footnote',
 'id',
 'max',
 'median',
 'min',
 'nutrient',
 'type']

🔸 Pola w nutrient:
['id', 'name', 'number', 'rank', 'unitName']

🔸 Pola w foodNutrientDerivation:
['code', 'description', 'foodNutrientSource']

🔸 Pola w foodNutrientDerivation.foodNutrientSource:
['code', 'description', 'id']


In [5]:
def extract_paths(obj, path=""):
    paths = set()

    if isinstance(obj, dict):
        for k, v in obj.items():
            new_path = f"{path}.{k}" if path else k
            paths.update(extract_paths(v, new_path))
    elif isinstance(obj, list):
        for item in obj:
            paths.update(extract_paths(item, path))
    else:
        paths.add(path)

    return paths

# Przetwarzamy dane z jednego przykładowego produktu
sample_food = data["FoundationFoods"][0]

# Zbieramy wszystkie ścieżki
all_paths = extract_paths(sample_food)

# Wypisujemy posortowaną listę
for p in sorted(all_paths):
    print(p)


dataType
description
fdcId
foodCategory.description
foodClass
foodNutrients.amount
foodNutrients.dataPoints
foodNutrients.foodNutrientDerivation.code
foodNutrients.foodNutrientDerivation.description
foodNutrients.foodNutrientDerivation.foodNutrientSource.code
foodNutrients.foodNutrientDerivation.foodNutrientSource.description
foodNutrients.foodNutrientDerivation.foodNutrientSource.id
foodNutrients.id
foodNutrients.max
foodNutrients.median
foodNutrients.min
foodNutrients.nutrient.id
foodNutrients.nutrient.name
foodNutrients.nutrient.number
foodNutrients.nutrient.rank
foodNutrients.nutrient.unitName
foodNutrients.type
foodPortions.amount
foodPortions.gramWeight
foodPortions.id
foodPortions.measureUnit.abbreviation
foodPortions.measureUnit.id
foodPortions.measureUnit.name
foodPortions.minYearAcquired
foodPortions.modifier
foodPortions.sequenceNumber
foodPortions.value
inputFoods.foodDescription
inputFoods.id
inputFoods.inputFood.dataType
inputFoods.inputFood.description
inputFoods.inputFo

## Krótki opis etykiet na podstawie dokumentacji
#### link do doku: https://fdc.nal.usda.gov/Foundation_Foods_Documentation


---

## 🔹 Główne informacje o produkcie

| Pole | Znaczenie |
|------|-----------|
| `dataType` | Typ danych w FDC (np. `Foundation`, `SR Legacy`, `Branded`). |
| `description` | Pełna nazwa / opis żywności. |
| `fdcId` | Unikalny identyfikator rekordu w FoodData Central. |
| `foodCategory.description` | Kategoria żywności (historyczny podział z SR Legacy). |
| `foodClass` | Klasa rekordu – np. `FinalFood` (gotowy produkt) albo `Ingredient`. |

---

## 🔹 Wartości odżywcze `foodNutrients[]`

| Pole | Znaczenie |
|------|-----------|
| `foodNutrients.amount` | Zawartość składnika w 100 g produktu. |
| `foodNutrients.dataPoints` | Liczba niezależnych pomiarów użytych w analizie. |
| `foodNutrients.id` | Wewnętrzny klucz rekordu składnika. |
| `foodNutrients.type` | Typ rekordu (zwykle `FoodNutrient`). |
| `foodNutrients.max` / `min` / `median` | Statystyki wartości wśród próbek. |

**Wewnątrz `nutrient`:**

| Pole | Znaczenie |
|------|-----------|
| `foodNutrients.nutrient.id` | Kod składnika odżywczego (np. 1003 = białko). |
| `foodNutrients.nutrient.name` | Nazwa składnika (np. „Vitamin C”). |
| `foodNutrients.nutrient.number` | Numer USDA – zgodny z dawnym SR. |
| `foodNutrients.nutrient.rank` | Kolejność wyświetlania w interfejsie FDC. |
| `foodNutrients.nutrient.unitName` | Jednostka miary (g, mg, µg …). |

**Wewnątrz `foodNutrientDerivation`:**

| Pole | Znaczenie |
|------|-----------|
| `foodNutrients.foodNutrientDerivation.code` | Kod źródła danych (`A` = analiza lab., `AS` = suma). |
| `foodNutrients.foodNutrientDerivation.description` | Opis sposobu uzyskania wartości (Analytical, Estimated…). |

**Wewnątrz `foodNutrientSource`:**

| Pole | Znaczenie |
|------|-----------|
| `foodNutrientDerivation.foodNutrientSource.id` | ID pierwotnego źródła pomiaru. |
| `…code` / `…description` | Kod i opis źródła (np. „Analytical or derived from analytical”). |

---

## 🔹 Porcje `foodPortions[]`

| Pole | Znaczenie |
|------|-----------|
| `foodPortions.id` | ID rekordu porcji. |
| `foodPortions.amount` | Liczba jednostek (np. 1 szt.). |
| `foodPortions.value` / `gramWeight` | Masa odpowiadająca porcji (g). |
| `foodPortions.modifier` | Opis wielkości („1 cup”, „large slice”…). |
| `foodPortions.sequenceNumber` | Kolejność porcji w rekordzie. |
| `foodPortions.minYearAcquired` | Rok pozyskania próbki dla tej porcji. |

**Jednostka miary (`measureUnit`):**

| Pole | Znaczenie |
|------|-----------|
| `measureUnit.id` | ID jednostki (np. 999 = gram). |
| `measureUnit.name` | Nazwa („cup”, „gram”…). |
| `measureUnit.abbreviation` | Skrót („g”, „c”…). |

---

## 🔹 Skład produktu `inputFoods[]`

| Pole | Znaczenie |
|------|-----------|
| `inputFoods.id` | ID pozycji w recepturze. |
| `inputFoods.foodDescription` | Opis składnika (tekstowy). |
| `inputFoods.inputFood.fdcId` | FDC ID składnika wejściowego. |
| `inputFoods.inputFood.description` | Nazwa składnika. |
| `inputFoods.inputFood.dataType` | Typ danych składnika (Foundation, Branded…). |
| `inputFoods.inputFood.foodClass` | Klasa składnika (Ingredient, FinalFood…). |
| `inputFoods.inputFood.publicationDate` | Data publikacji rekordu składnika. |
| `inputFoods.inputFood.foodCategory.*` | Kategoria składnika (kod, opis, id). |

---

## 🔹 Współczynniki kaloryczne `nutrientConversionFactors[]`

| Pole | Znaczenie |
|------|-----------|
| `type` | Rodzaj współczynnika (`.CalorieConversionFactor`, `.ProteinConversionFactor` …). |
| `proteinValue` / `fatValue` / `carbohydrateValue` | Współczynniki kcal/g dla makroskładników. |
| `value` | Uniwersalna wartość, gdy istnieje tylko jeden współczynnik. |

---

## 🔹 Pola dodatkowe

| Pole | Znaczenie |
|------|-----------|
| `isHistoricalReference` | `True`, jeśli rekord pochodzi z archiwalnego SR Legacy. |
| `ndbNumber` | Dawny numer w bazie SR Legacy (kompatybilność). |
| `publicationDate` | Data publicznej publikacji rekordu w FDC. |




## co defacto zawiera "food portions"

Każdy produkt w Foundation Foods może mieć **kilka rekordów porcji**.  
Każdy rekord zawiera dane potrzebne do przeliczenia makroskładników na realne porcje.

### 📋 Przykład porcji – banan vs jajko

| Klucz w JSON-ie       | Co oznacza w praktyce                                  | Przykład banana                   | Przykład jajka                      |
|------------------------|----------------------------------------------------------|------------------------------------|-------------------------------------|
| `measureUnit`          | Słownikowa jednostka (`gram`, `cup`, `item`…)           | `"item"` (sztuka)                  | `"item"` (sztuka)                   |
| `value`                | „1 × measureUnit” – czyli rozmiar tej jednostki         | `1.0`                              | `1.0`                               |
| `amount`               | Ile takich jednostek opisuje porcja                     | `1.0` (1 szt.)<br>lub `0.5` (połówka) | `1.0`                               |
| `modifier`             | Opis wielkości/kształtu                                 | `"medium"` / `"large"`             | `"medium"` / `"large"`              |
| `gramWeight`           | Masa jadalnej części porcji (bez skórki/skorupki)       | `120 g` (średni banan bez skórki)  | `50 g` (duże jajko bez skorupki)    |

> Wystarczy znać `gramWeight` i przeliczyć składniki według wzoru:  
> `wartość_porcji = wartość_na_100g × (gramWeight / 100)`


In [7]:
unit_names = set()
unit_abbr  = set()
unit_ids   = set()

for food in data["FoundationFoods"]:
    for p in food.get("foodPortions", []):
        mu = p.get("measureUnit", {})
        unit_names.add(mu.get("name"))
        unit_abbr.add(mu.get("abbreviation"))
        unit_ids.add(mu.get("id"))

print("Unikalne measureUnit.name:")
pprint(sorted(filter(None, unit_names)))

print("\nUnikalne measureUnit.abbreviation:")
pprint(sorted(filter(None, unit_abbr)))

print("\nUnikalne measureUnit.id:")
pprint(sorted(filter(None, unit_ids)))

Unikalne measureUnit.name:
['Banana',
 'Onion',
 'bunch',
 'container',
 'cookie',
 'cup',
 'drumstick',
 'each',
 'egg',
 'fillet',
 'fruit',
 'link',
 'milliliter',
 'olive',
 'order',
 'oz',
 'package',
 'paired cooked w',
 'piece',
 'pieces',
 'roast',
 'serving',
 'slice',
 'spear',
 'steak',
 'tablespoon',
 'teaspoon',
 'tomatoes',
 'wedge']

Unikalne measureUnit.abbreviation:
['Banana',
 'Onion',
 'bunch',
 'ckd pr g',
 'container',
 'cookie',
 'cup',
 'drumstick',
 'each',
 'egg',
 'fillet',
 'fruit',
 'link',
 'ml',
 'olive',
 'order',
 'oz',
 'package',
 'piece',
 'pieces',
 'roast',
 'serving',
 'slice',
 'spear',
 'steak',
 'tbsp',
 'tomatoes',
 'tsp',
 'wedge']

Unikalne measureUnit.id:
[1000,
 1001,
 1002,
 1004,
 1010,
 1022,
 1023,
 1026,
 1027,
 1028,
 1033,
 1038,
 1039,
 1043,
 1044,
 1046,
 1049,
 1050,
 1054,
 1060,
 1067,
 1069,
 1071,
 1077,
 1082,
 1099,
 1117,
 1119,
 1120]


In [8]:
import json
import pandas as pd

# ---- 1. wczytanie danych ----------------------------------------------------
with open("raw_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)["FoundationFoods"]

# ---- 2. zdefiniuj szukane składniki -----------------------------------------
#  ID-y / nazwy wzięte wprost z dokumentacji Foundation Foods (IV-2024)
TARGETS = {
    "kcal": {
        "prefer_id": [2047, 2048, 1008],           # Atwater General, Specific, dawny "Energy"
        "prefer_name": [
            "Metabolizable Energy (Atwater General Factor)",
            "Metabolizable Energy (Atwater Specific Factor)",
            "Energy",
        ],
    },
    "protein_g": {
        "prefer_id": [1003],
        "prefer_name": ["Protein"],
    },
    "fat_g": {
        "prefer_id": [1004],
        "prefer_name": ["Total lipid (fat)"],
    },
    "carb_g": {
        "prefer_id": [1005],
        "prefer_name": ["Carbohydrate, by difference"],
    },
}

# ---- 3. funkcja pomocnicza ---------------------------------------------------
def pick_value(food_nutrients, prefer_id, prefer_name):
    """Zwraca amount dla pierwszego pasującego ID lub nazwy."""
    for pid in prefer_id:                       # priorytet ID
        for item in food_nutrients:
            if item["nutrient"]["id"] == pid:
                return item["amount"]
    for pname in prefer_name:                   # ewentualnie priorytet nazw
        for item in food_nutrients:
            if item["nutrient"]["name"] == pname:
                return item["amount"]
    return None                                 # brak w produkcie

# ---- 4. ekstrakcja danych ----------------------------------------------------
rows = []
for food in data:
    nutrients = food.get("foodNutrients", [])
    rows.append(
        {
            "fdcId": food["fdcId"],
            "description": food["description"],
            "kcal_per_100g": pick_value(nutrients, **TARGETS["kcal"]),
            "protein_g_per_100g": pick_value(nutrients, **TARGETS["protein_g"]),
            "fat_g_per_100g": pick_value(nutrients, **TARGETS["fat_g"]),
            "carb_g_per_100g": pick_value(nutrients, **TARGETS["carb_g"]),
        }
    )

df = pd.DataFrame(rows)
print(df.head())       


    fdcId                                        description  kcal_per_100g  \
0  321358                                 Hummus, commercial          229.0   
1  321360                               Tomatoes, grape, raw           27.0   
2  321611  Beans, snap, green, canned, regular pack, drai...           21.0   
3  323121                        Frankfurter, beef, unheated          314.0   
4  323294        Nuts, almonds, dry roasted, with salt added          620.0   

   protein_g_per_100g  fat_g_per_100g  carb_g_per_100g  
0                7.35           17.10            14.90  
1                0.83            0.63             5.51  
2                1.04            0.39             4.11  
3               11.70           28.00             2.89  
4               20.40           57.80            16.20  


#### sanity check, czy coś nie ma >900 kcal per 100 g

In [9]:
max_row = df.loc[df["kcal_per_100g"].idxmax()]
min_row = df.loc[df["kcal_per_100g"].idxmin()]

print("🍔 Najwięcej kcal/100 g:")
print(f'  {max_row["description"]} — {max_row["kcal_per_100g"]} kcal')

print("\n🥒 Najmniej kcal/100 g:")
print(f'  {min_row["description"]} — {min_row["kcal_per_100g"]} kcal')


🍔 Najwięcej kcal/100 g:
  Oil, coconut — 833.0 kcal

🥒 Najmniej kcal/100 g:
  Pickles, cucumber, dill or kosher dill — 12.0 kcal


#### Imo jest git

#### kolejny check, czy kcal per 100 gram zgadza się z wzorem kcal = fat * 9 + carbs * 4 + protein * 4

In [12]:
df["kcal_calculated"] = (
    df["fat_g_per_100g"] * 9
    + df["carb_g_per_100g"] * 4
    + df["protein_g_per_100g"] * 4
)

df["kcal_difference"] = abs(df["kcal_per_100g"] - df["kcal_calculated"])

df_incorrect = df[df["kcal_difference"] > 10]

# podejrzane rekordy to te, gdzie różnica między kcal w bazie a spodziewanymi wynosi ponad 10
df_incorrect[["description", "kcal_per_100g", "kcal_calculated", "kcal_difference"]]


,description,kcal_per_100g,kcal_calculated,kcal_difference
0,"Hummus, commercial",229.0,242.90,13.90
4,"Nuts, almonds, dry roasted, with salt added",620.0,666.60,46.60
8,"Egg, white, dried",376.0,349.53,26.47
15,"Seeds, sunflower seed kernels, dry roasted, wi...",612.0,657.30,45.30
24,"Egg, whole, dried",575.0,558.08,16.92
26,"Egg, yolk, dried",654.0,640.58,13.42
29,"Oil, coconut",833.0,895.26,62.26
34,"Olives, green, Manzanilla, stuffed with pimiento",130.0,140.54,10.54
54,"Figs, dried, uncooked",249.0,277.08,28.08
70,"Sugars, granulated",385.0,401.28,16.28


#### imo ma sens, prawie wszystkie produkty stosują się do wzoru, róznica może wynikać z tego, że wzrór 9 4 4 jest tylko przybliżeniem, np błonnik może mieć wpływ na te wyniki, jednkaże wygląda że kcal per 100 gram jest sensownie obliczone. 

#### Analiza wg czata:

#### sanity cehck wg wzoru kcal = fat × 9 + protein × 4 + carbs × 4

Moim zdaniem wartości `kcal per 100 g` mają sens – zdecydowana większość produktów dobrze zgadza się ze wzorem:

> `kcal ≈ tłuszcz × 9 + białko × 4 + węglowodany × 4`

Rozbieżności mogą wynikać z faktu, że ten wzór to tylko przybliżenie.  
USDA stosuje dokładniejsze przeliczniki (*Atwater specific factors*) i uwzględnia np.:

- błonnik pokarmowy (który może mieć 0–2 kcal/g),
- alkohole cukrowe (niższa kaloryczność niż zwykłe węgle),
- różnice w przyswajalności składników odżywczych.

Finalnie jedank dane mają sens


### Struktura DTO produktu w Springu

Na ten moment rekord `ProductDTO` w backendzie wygląda tak:

link dla swenda: https://github.com/Jeremylaby/Diet-Optimization-App/tree/feature/DIET-57-Product-MVC/dyeti-backend/src/main/java/ki/agh/dyeti/dto

```java
public record ProductDTO(
    Long id,
    String name,
    UnitDTO unit,
    Double gramsPerUnit,
    Double kcal100g,
    Double protein100g,
    Double carbs100g,
    Double fat100g,
    Long ownerId
)
```
Rekord UnitDTO wygląda następująco:

```java
public record UnitDTO(Long id, String name, String symbol) {
    public static UnitDTO fromEntity(Unit unit) {
        return new UnitDTO(unit.getId(), unit.getName(), unit.getSymbol());
    }
}
```

póki co owner nas nie obchodzi, unitDTO to nwm, chyba robimy nową tebele na unit? (???) ale niech  będzie. Kcal i makro per 100 gram już jest . Teraz zajme się tymii porcjami (unit itp)


### Jak wypełnić `unit` i `gramsPerUnit` z danych USDA (sekcja **Weights**)

| Co mówi dokumentacja Foundation Foods (sekcja *Weights*) | Co trafia do DTO |
|-----------------------------------------------------------|---------------------------|
| **`gramWeight`** = masa porcji w gramach „for edible material without refuse” – np. obrany banan, jajko bez skorupki. | → **`ProductDTO.gramsPerUnit`** |
| **`measureUnit.id`** – unikatowy identyfikator jednostki (własny słownik Foundation Foods). | → **`UnitDTO.id`** |
| **`measureUnit.name`** – nazwa jednostki („gram”, „cup”, „Egg”, „Banana”, „slice”…). | → **`UnitDTO.name`** |
| **`measureUnit.abbreviation`** – skrót / symbol („g”, „cup”, „egg”…). | → **`UnitDTO.symbol`** |
| Porcja jest opisana wartością **`value`** (zwykle 1.0) + opcjonalnym **`modifier`** („medium”, „large”). | • wybieramy rekord z `value == 1.0` <br>• w razie wielu – wariant `modifier == "medium"` lub pierwszy z listy |
| Formula w dokumencie: `N = (V × W) / 100` –  czyli makro na 100 g × (gramWeight / 100). | używamy jej, gdy użytkownik poda liczbę porcji (np. 2 jajka) |

> TL;DR  
> • `gramWeight` ⇒ `gramsPerUnit`  
> • `measureUnit.{id,name,abbreviation}` ⇒ `UnitDTO`  
> • w braku porcji fallback = *“100 g”* (100 g = 1 jednostka)  
> • dzięki temu konwersja makroskładników na 1 porcję = wzór z dokumentacji


In [13]:
import json
import pandas as pd

# 1. Wczytanie danych
with open("raw_data.json", "r", encoding="utf-8") as f:
    foods = json.load(f)["FoundationFoods"]

# 2. Definicja makroskładników
TARGETS = {
    "kcal":    {"prefer_id":[2047,2048,1008], "prefer_name":["Metabolizable Energy (Atwater General Factor)", "Metabolizable Energy (Atwater Specific Factor)", "Energy"]},
    "protein_g":{"prefer_id":[1003],         "prefer_name":["Protein"]},
    "fat_g":   {"prefer_id":[1004],         "prefer_name":["Total lipid (fat)"]},
    "carb_g":  {"prefer_id":[1005],         "prefer_name":["Carbohydrate, by difference"]},
}

def pick_value(nutrients, prefer_id, prefer_name):
    # zwraca 'amount' dla pierwszego pasującego ID, potem nazwy
    for pid in prefer_id:
        for x in nutrients:
            if x["nutrient"]["id"] == pid:
                return x.get("amount")
    for name in prefer_name:
        for x in nutrients:
            if x["nutrient"]["name"] == name:
                return x.get("amount")
    return None

def choose_portion(portions):
    # 1) medium value==1
    for p in portions:
        if p.get("value")==1 and p.get("modifier","").lower()=="medium":
            return p
    # 2) dowolne value==1
    for p in portions:
        if p.get("value")==1:
            return p
    # 3) fallback: 100 g
    return {
        "measureUnit": {"id": None, "name": "100g", "abbreviation": "g"},
        "gramWeight": 100.0
    }

# 3. Budowa listy wierszy
rows = []
for food in foods:
    nuts = food.get("foodNutrients", [])
    portion = choose_portion(food.get("foodPortions", []))
    mu = portion["measureUnit"]
    rows.append({
        "fdcId":               food.get("fdcId"),
        "description":         food.get("description"),
        "kcal_per_100g":       pick_value(nuts, **TARGETS["kcal"]),
        "protein_g_per_100g":  pick_value(nuts, **TARGETS["protein_g"]),
        "fat_g_per_100g":      pick_value(nuts, **TARGETS["fat_g"]),
        "carb_g_per_100g":     pick_value(nuts, **TARGETS["carb_g"]),
        "measureUnitName":     mu.get("name"),
        "measureUnitId":       mu.get("id"),
        "measureUnitSymbol":   mu.get("abbreviation"),
        "gramsPerUnit":        portion.get("gramWeight")
    })

# 4. Stworzenie DataFrame
df_final = pd.DataFrame(rows)

# 5. Podgląd
print(df_final.head())


    fdcId                                        description  kcal_per_100g  \
0  321358                                 Hummus, commercial          229.0   
1  321360                               Tomatoes, grape, raw           27.0   
2  321611  Beans, snap, green, canned, regular pack, drai...           21.0   
3  323121                        Frankfurter, beef, unheated          314.0   
4  323294        Nuts, almonds, dry roasted, with salt added          620.0   

   protein_g_per_100g  fat_g_per_100g  carb_g_per_100g measureUnitName  \
0                7.35           17.10            14.90            100g   
1                0.83            0.63             5.51             cup   
2                1.04            0.39             4.11             cup   
3               11.70           28.00             2.89           piece   
4               20.40           57.80            16.20             cup   

   measureUnitId measureUnitSymbol  gramsPerUnit  
0            NaN             

In [14]:
#dopasowałem nazwy aby sie zgadzało z DTO

df_final = df_final.rename(columns={
    "fdcId":              "usda_id",
    "description":        "name",
    "kcal_per_100g":      "kcal100g",
    "protein_g_per_100g": "protein100g",
    "fat_g_per_100g":     "fat100g",
    "carb_g_per_100g":    "carbs100g",
})

# Kolejność kolumn zgodna z ProductDTO (+ pola jednostki)
df_final = df_final[
    [
        "usda_id",
        "name",
        "measureUnitId",
        "measureUnitName",
        "measureUnitSymbol",
        "gramsPerUnit",
        "kcal100g",
        "protein100g",
        "carbs100g",
        "fat100g",
    ]
]

print(df_final.head())

   usda_id                                               name  measureUnitId  \
0   321358                                 Hummus, commercial            NaN   
1   321360                               Tomatoes, grape, raw         1000.0   
2   321611  Beans, snap, green, canned, regular pack, drai...         1000.0   
3   323121                        Frankfurter, beef, unheated         1043.0   
4   323294        Nuts, almonds, dry roasted, with salt added         1000.0   

  measureUnitName measureUnitSymbol  gramsPerUnit  kcal100g  protein100g  \
0            100g                 g         100.0     229.0         7.35   
1             cup               cup         152.0      27.0         0.83   
2             cup               cup         129.0      21.0         1.04   
3           piece             piece          48.6     314.0        11.70   
4             cup               cup         135.0     620.0        20.40   

   carbs100g  fat100g  
0      14.90    17.10  
1       5.51  

#### patrze, czy są braki w wartości

In [15]:

null_rows = df_final[df_final.isnull().any(axis=1)]

print(f"Liczba wierszy z brakami: {len(null_rows)}")
null_rows


Liczba wierszy z brakami: 264


,usda_id,name,measureUnitId,measureUnitName,measureUnitSymbol,gramsPerUnit,kcal100g,protein100g,carbs100g,fat100g
0,321358,"Hummus, commercial",NaN,100g,g,100.0,229.0,7.350,14.90,17.10
19,327046,"Kiwifruit, green, raw",NaN,100g,g,100.0,58.0,1.060,14.00,0.44
21,328637,"Cheese, cheddar",NaN,100g,g,100.0,408.0,23.300,2.44,34.00
39,334194,"Fish, tuna, light, canned in water, drained so...",NaN,100g,g,100.0,90.0,19.000,0.08,0.94
50,746764,"Carrots, frozen, unprepared",NaN,100g,g,100.0,37.0,0.810,7.92,0.47
...,...,...,...,...,...,...,...,...,...,...
335,2727585,"Green onion, (scallion), bulb and greens, root...",NaN,100g,g,100.0,NaN,0.669,NaN,NaN
336,2727586,"Shallots, bulb, peeled, root removed, raw",NaN,100g,g,100.0,NaN,1.380,NaN,NaN
337,2727587,"Juice, prune, shelf-stable",NaN,100g,g,100.0,NaN,0.423,NaN,NaN
338,2727588,"Juice, pomegranate, from concentrate, shelf-st...",NaN,100g,g,100.0,NaN,0.000,NaN,NaN


#### Ok, jest pewien problem, bardzo duża częśc produktów ma braki, przyjrzę się temu dokładniej

Najpier zajme sie tymi, gdzie USDA nie podało measure Unit

In [16]:
null_unit_rows = df_final[df_final["measureUnitId"].isna()]

print(f"Liczba wierszy z pustym measureUnitId: {len(null_unit_rows)}")
null_unit_rows[[
    "usda_id",
    "name",
    "measureUnitName",
    "gramsPerUnit",
    "kcal100g",
    "protein100g",
    "carbs100g",
    "fat100g"
]]


Liczba wierszy z pustym measureUnitId: 263


,usda_id,name,measureUnitName,gramsPerUnit,kcal100g,protein100g,carbs100g,fat100g
0,321358,"Hummus, commercial",100g,100.0,229.0,7.350,14.90,17.10
19,327046,"Kiwifruit, green, raw",100g,100.0,58.0,1.060,14.00,0.44
21,328637,"Cheese, cheddar",100g,100.0,408.0,23.300,2.44,34.00
39,334194,"Fish, tuna, light, canned in water, drained so...",100g,100.0,90.0,19.000,0.08,0.94
50,746764,"Carrots, frozen, unprepared",100g,100.0,37.0,0.810,7.92,0.47
...,...,...,...,...,...,...,...,...
335,2727585,"Green onion, (scallion), bulb and greens, root...",100g,100.0,NaN,0.669,NaN,NaN
336,2727586,"Shallots, bulb, peeled, root removed, raw",100g,100.0,NaN,1.380,NaN,NaN
337,2727587,"Juice, prune, shelf-stable",100g,100.0,NaN,0.423,NaN,NaN
338,2727588,"Juice, pomegranate, from concentrate, shelf-st...",100g,100.0,NaN,0.000,NaN,NaN


In [17]:
mask_na = df_final["measureUnitId"].isna()

df_final.loc[mask_na, "measureUnitId"]     = 2137
df_final.loc[mask_na, "measureUnitName"]   = "100g"
df_final.loc[mask_na, "measureUnitSymbol"] = "g"
df_final.loc[mask_na, "gramsPerUnit"]      = 100.0  # na wszelki wypadek

print(f"Podmieniono {mask_na.sum()} wierszy na unitId = 2137 (100 g).")


Podmieniono 263 wierszy na unitId = 2137 (100 g).


#### unitID w USDA mieszą się miedzy 1000 a 2000, więc tymczasowo dałem nasz customowy placeholder "2137" żeby się zgadzało z DTO

In [18]:
null_rows = df_final[df_final.isnull().any(axis=1)]

print(f"Liczba wierszy z brakami: {len(null_rows)}")
null_rows[
    [
        "usda_id",
        "name",
        "measureUnitId",
        "measureUnitName",
        "measureUnitSymbol",
        "gramsPerUnit",
        "kcal100g",
        "protein100g",
        "carbs100g",
        "fat100g",
    ]
]

Liczba wierszy z brakami: 41


,usda_id,name,measureUnitId,measureUnitName,measureUnitSymbol,gramsPerUnit,kcal100g,protein100g,carbs100g,fat100g
61,746775,"Salt, table, iodized",1002.0,teaspoon,tsp,6.1,NaN,NaN,NaN,NaN
74,747430,"Beans, Dry, Medium Red (0% moisture)",2137.0,100g,g,100.0,NaN,25.500,NaN,1.04
75,747431,"Beans, Dry, Red (0% moisture)",2137.0,100g,g,100.0,NaN,21.300,NaN,1.16
76,747432,"Beans, Dry, Flor de Mayo (0% moisture)",2137.0,100g,g,100.0,NaN,23.300,NaN,0.86
77,747433,"Beans, Dry, Brown (0% moisture)",2137.0,100g,g,100.0,NaN,25.600,NaN,1.12
78,747434,"Beans, Dry, Tan (0% moisture)",2137.0,100g,g,100.0,NaN,26.800,NaN,1.14
79,747435,"Beans, Dry, Light Tan (0% moisture)",2137.0,100g,g,100.0,NaN,24.600,NaN,1.28
80,747436,"Beans, Dry, Carioca (0% moisture)",2137.0,100g,g,100.0,NaN,25.200,NaN,1.44
81,747437,"Beans, Dry, Cranberry (0% moisture)",2137.0,100g,g,100.0,NaN,24.400,NaN,1.23
82,747438,"Beans, Dry, Light Red Kidney (0% moisture)",2137.0,100g,g,100.0,NaN,25.000,NaN,1.03


Dobra, teraz mamy tylko 41 produktów z barkujacymi danymi, jednakże te z brakami nie mają fat, carbs lub fat, więc niestety są bezużyteczne dla naszego celu, dlatego na ten momen zostaną pominiete i nie znajdą sie w gotowych danych w json'ie 

In [19]:
import json
import pandas as pd
import os

print("Bieżący katalog:", os.getcwd())  

df_ready = df_final.dropna()              # odrzucam wiersze z brakami

df_ready.to_json("ready_data.json", orient="records", indent=2)

print("✅ Zapisano", len(df_ready), "rekordów do pliku ready_data.json")


Bieżący katalog: C:\Users\piotr\inzynierka_usda\dYeti_data\jowisz
✅ Zapisano 299 rekordów do pliku ready_data.json


### Gotowe rekordy do importu – szybki podgląd

| usda_id | name                       | unit (id · name · symbol) | grams / unit | kcal / 100 g | P g/100 g | C g/100 g | F g/100 g |
|---------|---------------------------|---------------------------|--------------|--------------|-----------|-----------|-----------|
| 323121  | Frankfurter, beef, unheated | 1043 · piece · piece      | **48.6 g**   | 314          | 11.7      | 2.89      | 28.0      |
| 321360  | Tomatoes, grape, raw        | 1000 · cup · cup          | **152 g**    | 27           | 0.83      | 5.51      | 0.63      |
| 328637  | Cheese, cheddar             | 2137 · 100g · g           | **100 g**    | 408          | 23.3      | 2.44      | 34.0      |

**Co widać?**

* Każdy produkt ma komplet pól wymaganych przez `ProductDTO`.  
* Jednostka („piece”, „cup”, „100 g”) ma swój unikalny `measureUnitId` oraz `gramsPerUnit`.  
* Kalorie i makro odnoszą się do 100 g jadalnej części, zgodnie z danymi USDA.

Plik **`ready_data.json`** zawiera analogiczne obiekty dla wszystkich produktów bez braków (`NaN`).


### Przykładowe rekordy z `ready_data.json`

```json
{
  "usda_id": 323121,
  "name": "Frankfurter, beef, unheated",
  "measureUnitId": 1043,
  "measureUnitName": "piece",
  "measureUnitSymbol": "piece",
  "gramsPerUnit": 48.6,
  "kcal100g": 314.0,
  "protein100g": 11.7,
  "carbs100g": 2.89,
  "fat100g": 28.0
},
{
  "usda_id": 321360,
  "name": "Tomatoes, grape, raw",
  "measureUnitId": 1000,
  "measureUnitName": "cup",
  "measureUnitSymbol": "cup",
  "gramsPerUnit": 152.0,
  "kcal100g": 27.0,
  "protein100g": 0.83,
  "carbs100g": 5.51,
  "fat100g": 0.63
},
{
  "usda_id": 328637,
  "name": "Cheese, cheddar",
  "measureUnitId": 2137,
  "measureUnitName": "100g",
  "measureUnitSymbol": "g",
  "gramsPerUnit": 100.0,
  "kcal100g": 408.0,
  "protein100g": 23.3,
  "carbs100g": 2.44,
  "fat100g": 34.0
}
